In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset


tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-135M")
model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-135M")

data = load_dataset("Ashed00/combined_math_problems", split="train")
data

Dataset({
    features: ['ID', 'Body', 'Question', 'Equation', 'Type', 'question', 'answer', 'source', 'input'],
    num_rows: 6320
})

In [2]:
# prompt: I want to remove all columns with source AddSub

addsub_data = data.filter(lambda x: x['source'] == 'AddSub')
data = data.filter(lambda x: x['source'] != 'AddSub')


#select 200 random rows from addsub

addsub_data = addsub_data.shuffle().select(range(200))

print(addsub_data)

#concat addsub with data
from datasets import concatenate_datasets
#data = concatenate_datasets([data, addsub_data])
data

Dataset({
    features: ['ID', 'Body', 'Question', 'Equation', 'Type', 'question', 'answer', 'source', 'input'],
    num_rows: 200
})


Dataset({
    features: ['ID', 'Body', 'Question', 'Equation', 'Type', 'question', 'answer', 'source', 'input'],
    num_rows: 1120
})

In [3]:
data = data.to_pandas()

# Drop all columns except 'question' and 'answer'
columns_to_keep = ['question', 'answer']
columns_to_drop = [col for col in data.columns if col not in columns_to_keep]
data = data.drop(columns=columns_to_drop)

# Drop rows with null values in 'question' or 'answer'
data = data.dropna(subset=['question', 'answer'])

# Drop rows where 'question' or 'answer' are not strings
data = data[data['question'].apply(lambda x: isinstance(x, str))]
data = data[data['answer'].apply(lambda x: isinstance(x, str))]

from datasets import Dataset
data = Dataset.from_pandas(data)

data = data.shuffle()

data


Dataset({
    features: ['question', 'answer'],
    num_rows: 1120
})

In [4]:
data[1000]

{'question': '21 children were riding on the bus. At the bus stop 10 children got off the bus while some more got on the bus. Then there were 16 children altogether on the bus. How many children got on the bus at the bus stop?',
 'answer': '5'}

In [5]:
# prompt: Update question to the form "Q: question /n A:"

def format_qa(example):
  example['question'] = "Question: "+f"{example['question']}".strip()+"\n Answer:"
  example['answer'] = f"{example['answer']}".strip() +"\n#End of Answer." + str(tokenizer.eos_token)
  return example

data = data.map(format_qa)

data[1000]


Map:   0%|          | 0/1120 [00:00<?, ? examples/s]

{'question': 'Question: 21 children were riding on the bus. At the bus stop 10 children got off the bus while some more got on the bus. Then there were 16 children altogether on the bus. How many children got on the bus at the bus stop?\n Answer:',
 'answer': '5\n#End of Answer.<|endoftext|>'}

In [6]:
# prompt: rename question to prompt and answer to completion

# Rename 'question' to 'prompt' and 'answer' to 'completion'
data = data.rename_column("question", "prompt")
data = data.rename_column("answer", "completion")

data

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 1120
})

In [ ]:
from trl import SFTConfig, SFTTrainer
from transformers import TrainingArguments, Trainer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Using device for training: {device}")

sp_data = data.train_test_split(test_size=0.1)

train_dataset = sp_data["train"]
eval_dataset = sp_data["test"]

tokenizer.pad_token = tokenizer.eos_token

training_args = TrainingArguments(
    output_dir="./smolmath-sft1", # Output directory
    num_train_epochs=8, # Number of training epochs
    per_device_train_batch_size=8, # Batch size per device during training
    save_steps=10_000, # Save checkpoint every X updates steps
    save_total_limit=2, # Limit the total amount of checkpoints
    logging_dir="./logs", # Directory for storing logs
    logging_steps=200,
    learning_rate=3e-5,
    weight_decay=0.01,
    eval_strategy="steps", # Evaluate every X steps
    eval_steps=200,
    # use_cpu = True if device.type == 'cpu' else False # Explicitly set use_cpu
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,
    args=training_args,
)

trainer.train()

trainer.save_model("./SmolMath0-SFT-NoCoT")

Using device for training: cuda


Map:   0%|          | 0/1008 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/1008 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/1008 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1008 [00:00<?, ? examples/s]

...

Applying chat template to eval dataset:   0%|          | 0/112 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/112 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/112 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: bt22ece049 (bt21ece003-nit-nagpur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
200,1.363900,1.073091
400,0.765800,0.879172
600,0.551000,0.841619
800,0.451600,0.863876
1000,0.393800,0.884131


In [8]:
trainer.save_model("./SmolMath0-SFT-NoCoT")

In [9]:
data[91]

{'prompt': "Question: Nancy's old washing machine could only wash 9 pieces of clothing at a time. If she had to wash 19 shirts and 8 sweaters how many loads would she have to do?\n Answer:",
 'completion': '3\n#End of Answer.<|endoftext|>'}

In [12]:
# prompt: Generate some text from model

# Encode the input prompt
input_text = "Question: Nancy's old washing machine could only wash 9 pieces of clothing at a time. If she had to wash 19 shirts and 8 sweaters how many loads would she have to do?\n Answer:"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

# Generate text
output = model.generate(input_ids, max_new_tokens=50, num_return_sequences=1, do_sample=True,top_k=50, temperature=0.6, pad_token_id=tokenizer.eos_token_id,)

# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
generated_text

"Question: Nancy's old washing machine could only wash 9 pieces of clothing at a time. If she had to wash 19 shirts and 8 sweaters how many loads would she have to do?\n Answer:8\n#End of Answer.\n#End of 2 statement question.\n#End ofhtaking question.\n#End of question frame.\n#End of melodie.\n#End of melodie.\n#End of"

In [11]:
# Evaluation function
from tqdm import tqdm
import re
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

tokenizer.pad_token = tokenizer.eos_token

# Helper: Extract last number (int or decimal) from string
def extract_last_number(text):
    numbers = re.findall(r'-?\d+(?:\.\d+)?', text)
    return numbers[-1] if numbers else None

# Evaluation function
def evaluate_accuracy(dataset, max_samples=10000):
    correct = 0
    total = 0

    for example in tqdm(dataset.select(range(min(len(dataset), max_samples)))):
        question = example['prompt']
        expected_answer = str(example['completion'])

        # Encode and generate
        inputs = tokenizer(question, return_tensors="pt").to(device)
        outputs = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)
        generated = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract answers
        generated_answer = generated[len(question):].strip()
        gen_number = extract_last_number(generated_answer)
        exp_number = extract_last_number(expected_answer)

        if gen_number is not None and exp_number is not None and gen_number == exp_number:
            correct += 1
        total += 1

    accuracy = correct / total if total > 0 else 0.0
    print(f"Accuracy (last number match): {accuracy:.2%}")

# Run evaluation
evaluate_accuracy(data)

100%|██████████| 1120/1120 [16:10<00:00,  1.15it/s]

Accuracy (last number match): 15.89%


In [14]:
from huggingface_hub import login

login()


In [15]:
model.push_to_hub("Ashed00/SmolMath-zero-NoCoT")
tokenizer.push_to_hub("Ashed00/SmolMath-zero-NoCoT")


model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Ashed00/SmolMath-zero-NoCoT/commit/f98d8c90e57aa069c60020a1e79221cbe5a2cfe6', commit_message='Upload tokenizer', commit_description='', oid='f98d8c90e57aa069c60020a1e79221cbe5a2cfe6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Ashed00/SmolMath-zero-NoCoT', endpoint='https://huggingface.co', repo_type='model', repo_id='Ashed00/SmolMath-zero-NoCoT'), pr_revision=None, pr_num=None)